In [1]:
pip install keras_nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 142 kB 10.7 MB/s 
     |████████████████████████████████| 5.9 MB 52.8 MB/s 
     |████████████████████████████████| 578.0 MB 18 kB/s 
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
     |████████████████████████████████| 4.6 MB 55.2 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import numpy as np

In [3]:
crime_and_punishment_url = 'https://www.gutenberg.org/files/2554/2554-0.txt'
brothers_of_karamazov_url = 'https://www.gutenberg.org/files/28054/28054-0.txt'
the_idiot_url = 'https://www.gutenberg.org/files/2638/2638-0.txt'
the_possessed_url = 'https://www.gutenberg.org/files/8117/8117-0.txt'

paths = [crime_and_punishment_url, brothers_of_karamazov_url, the_idiot_url, the_possessed_url]
names = ['Crime and Punishment', 'Brothers of Karamazov', 'The Idiot', 'The Possessed']
texts = ''
for index, path in enumerate(paths):
    filepath = keras.utils.get_file(f'{names[index]}.txt', origin=path)
    text = ''
    with open(filepath, encoding='utf-8') as f:
        text = f.read()
        # First 50 lines are the Gutenberg intro and preface
        # Skipping first 10k characters for each book should be approximately
        # removing the intros and prefaces.
        texts += text[10000:]

1483181/1483181 [==============================] - 0s 0us/step


In [4]:
text_list = texts.split('.')
text_list = list(filter(None, text_list))

import random
random.shuffle(text_list)
length = len(text_list)
#split the dataset 
text_train = text_list[:int(0.7*length)]
text_test = text_list[int(0.7*length):int(0.85*length)]
text_valid = text_list[int(0.85*length):]

In [5]:
from tensorflow.keras.layers import TextVectorization

#def custom_standardization(input_string):
#    sentence = tf.strings.lower(input_string)
#    sentence = tf.strings.regex_replace(sentence, "\n", " ")
#    return sentence

maxlen = 50
# You can also set calculate the longest sentence in the data - 25 in this case
# maxlen = len(max(text_list).split(' ')) 

vectorize_layer = TextVectorization(
#    standardize = custom_standardization,
    output_mode="int",
    output_sequence_length=maxlen + 1,
)

vectorize_layer.adapt(text_list)
vocab = vectorize_layer.get_vocabulary()
vocab_size = len(vocab)
index_lookup = dict(zip(range(len(vocab)), vocab))   
#example vectorrizing words
print(vectorize_layer(['hello world!']))


tf.Tensor(
[[  1 319   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]], shape=(1, 51), dtype=int64)


In [6]:
#DataSet creation
def create_dataset(text, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices(text)
  dataset = dataset.shuffle(buffer_size=256)
  dataset = dataset.batch(batch_size)
  return dataset 

def preprocess_text(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y

batch_size = 64

train_dataset = create_dataset(text_train, batch_size)

valid_dataset = create_dataset(text_valid, batch_size)

test_dataset = create_dataset(text_test, batch_size)

#preprocess text
train_dataset = train_dataset.map(preprocess_text)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.map(preprocess_text)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

valid_dataset = valid_dataset.map(preprocess_text)
valid_dataset = valid_dataset.prefetch(tf.data.AUTOTUNE)


In [7]:
embed_dim = 128
num_heads = 4

def create_model():
    inputs = keras.layers.Input(shape=(maxlen,), dtype=tf.int32)
    embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(vocab_size, maxlen, embed_dim)(inputs)
    decoder = keras_nlp.layers.TransformerDecoder(intermediate_dim=embed_dim, 
                                                            num_heads=num_heads, 
                                                            dropout=0.5)(embedding_layer)
    
    outputs = keras.layers.Dense(vocab_size, activation='softmax')(decoder)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer="adam", 
        loss='sparse_categorical_crossentropy',
        metrics=[keras_nlp.metrics.Perplexity(), 'accuracy']
    )
    return model

model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 token_and_position_embeddin  (None, 50, 128)          3861888   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, 50, 128)          99584     
 ormerDecoder)                                                   
                                                                 
 dense (Dense)               (None, 50, 30121)         3885609   
                                                                 
Total params: 7,847,081
Trainable params: 7,847,081
Non-trainable params: 0
___________________________________________________

In [8]:
#Custom Callback
class TextSampler(keras.callbacks.Callback):
    def __init__(self, start_prompt, max_tokens):
        self.start_prompt = start_prompt
        self.max_tokens = max_tokens
        
    # Helper method to choose a word from the top K probable words with respect to their probabilities
    # in a sequence
    def sample_token(self, logits):
        logits, indices = tf.math.top_k(logits, k=5, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def on_epoch_end(self, epoch, logs=None):
        decoded_sample = self.start_prompt
        
        for i in range(self.max_tokens-1):
            tokenized_prompt = vectorize_layer([decoded_sample])[:, :-1]
            predictions = self.model.predict([tokenized_prompt], verbose=0)
            # To find the index of the next word in the prediction array.
            # The tokenized prompt is already shorter than the original decoded sample
            # by one, len(decoded_sample.split()) is two words ahead - so we remove 1 to get
            # the next word in the sequence
            sample_index = len(decoded_sample.strip().split())-1
            
            sampled_token = self.sample_token(predictions[0][sample_index])
            sampled_token = index_lookup[sampled_token]
            decoded_sample += " " + sampled_token
            
        print(f"\nSample text:\n{decoded_sample}...\n")

# First 5 words of a random sentence to be used as a seed
random_sentence = ' '.join(random.choice(text_valid).replace('\n', ' ').split(' ')[:4])
sampler = TextSampler(random_sentence, 30)
reducelr = keras.callbacks.ReduceLROnPlateau(patience=10, monitor='val_loss')

In [9]:
model = create_model()
history = model.fit(train_dataset, 
                    validation_data=valid_dataset,
                    epochs=3, 
                    callbacks=[sampler, reducelr])

Epoch 1/3
633/634 [============================>.] - ETA: 0s - loss: 2.4861 - perplexity: 12.0139 - accuracy: 0.6826
Sample text:
 Oh, well, we know what is that you  are all that he would not be a little and he would come  for you are a very good thing and not...

634/634 [==============================] - 83s 122ms/step - loss: 2.4861 - perplexity: 12.0142 - accuracy: 0.6826 - val_loss: 1.9596 - val_perplexity: 7.0964 - val_accuracy: 0.6972 - lr: 0.0010
Epoch 2/3
633/634 [============================>.] - ETA: 0s - loss: 1.8398 - perplexity: 6.2953 - accuracy: 0.7034
Sample text:
 Oh, well, we are you  to the other people who have come to make up my own way i shall not at last night    to the first ...

634/634 [==============================] - 79s 125ms/step - loss: 1.8398 - perplexity: 6.2954 - accuracy: 0.7034 - val_loss: 1.9034 - val_perplexity: 6.7085 - val_accuracy: 0.7001 - lr: 0.0010
Epoch 3/3
633/634 [============================>.] - ETA: 0s - loss: 1.7248 - perplexity: 5.

In [10]:
def sample_token(logits):
        logits, indices = tf.math.top_k(logits, k=5, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

def generate_text(prompt, response_length=20):
    decoded_sample = prompt
    for i in range(response_length-1):
        tokenized_prompt = vectorize_layer([decoded_sample])[:, :-1]
        predictions = model.predict([tokenized_prompt], verbose=0)
        sample_index = len(decoded_sample.strip().split())-1

        sampled_token = sample_token(predictions[0][sample_index])
        sampled_token = index_lookup[sampled_token]
        decoded_sample += " " + sampled_token
    return decoded_sample

In [20]:
print(generate_text('the old woman '))
print(generate_text('The truth is '))

the old woman  of the prince was very fond to her in the prince   and that i should not know
The truth is  the other  people who can be no doubt of it and i know  what a great excitement
